<a href="https://colab.research.google.com/github/123nol/AI-projects/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
# from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
def vectorizeDataSet(dataset):
  dataset_copy=dataset.copy()
  sex_type=dataset_copy['sex'].unique()
  sex_lookup={x:idx for idx,x in enumerate(sex_type)}
  dataset_copy['int_sex']=dataset_copy['sex'].map(sex_lookup)
  smoker_lookup={"yes":1, "no":0 }
  dataset_copy['int_smoker']=dataset_copy['smoker'].map(smoker_lookup)
  unique_region=dataset_copy['region'].unique()
  region_lookup={x:idx for idx,x in enumerate(unique_region)}
  dataset_copy['int_region']=dataset_copy['region'].map(region_lookup)
  dataset_copy.drop(['sex','smoker','region'], axis=1, inplace=True)
  Y_train=dataset_copy['expenses']
  X_train=dataset_copy.drop(['expenses'],axis=1).values
  x_train,x_test,y_train,y_test=train_test_split(X_train,Y_train,test_size=0.2,random_state=35)
  training_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(buffer_size=1000).batch(32)
  test_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test)).shuffle(buffer_size=1000).batch(32)
  return training_dataset,test_dataset,dataset_copy,x_test,y_test,X_train,x_train,y_train







In [ ]:


training_dataset,whole_test_dataset,dataset_copy,test_dataset,test_labels,x_train,train_ds,train_label=vectorizeDataSet(dataset)
normalizer=tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(x_train)

model=tf.keras.Sequential([
        tf.keras.layers.Input(shape=(6,)),
        normalizer,
        tf.keras.layers.Dense(64,activation="relu")
        tf.keras.layers.Dense(32,activation="relu")

        tf.keras.layers.Dense(units=1,activation=None)
]
)

model.compile(
   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error',
    metrics=['mae','mse']
    )
model.summary()



history=model.fit(
    train_ds,
    train_label,
    batch_size=32,


    epochs=20
)




In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
